# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4872, uptime 0:00:33
mini_beamline                    RUNNING   pid 4873, uptime 0:00:33
random_walk                      RUNNING   pid 4874, uptime 0:00:33
random_walk_horiz                RUNNING   pid 4875, uptime 0:00:33
random_walk_vert                 RUNNING   pid 4876, uptime 0:00:33
simple                           RUNNING   pid 4877, uptime 0:00:33
thermo_sim                       RUNNING   pid 4878, uptime 0:00:33
trigger_with_pc                  RUNNING   pid 4879, uptime 0:00:33


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 3     Time: 2020-06-05 15:43:50
Persistent Unique Scan ID: 'd8ae0bbb-7eff-4c9c-b69d-eaae176bcc46'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 15:43:50.4 |     -1.000 |     100483 |
|         2 | 15:43:50.5 |     -0.500 |     101003 |
|         3 | 15:43:50.5 |      0.000 |     102432 |
|         4 | 15:43:50.5 |      0.500 |     101018 |
|         5 | 15:43:50.5 |      1.000 |      99833 |


+-----------+------------+------------+------------+
generator scan ['d8ae0bbb'] (scan num: 3)





('d8ae0bbb-7eff-4c9c-b69d-eaae176bcc46',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/d8ae0bbb-7eff-4c9c-b69d-eaae176bcc46-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 4     Time: 2020-06-05 15:43:50
Persistent Unique Scan ID: '1c6c14ee-c535-48dd-a439-71b6ce3f7412'


New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 15:43:50.9 |          -1 | 1087286498 |


|         2 | 15:43:51.1 |          -0 | 1079961456 |


|         3 | 15:43:51.2 |           0 | 1068855855 |


|         4 | 15:43:51.3 |           0 | 1064117220 |


|         5 | 15:43:51.4 |           1 | 1059395309 |


+-----------+------------+-------------+------------+
generator scan ['1c6c14ee'] (scan num: 4)





('1c6c14ee-c535-48dd-a439-71b6ce3f7412',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 5     Time: 2020-06-05 15:43:51
Persistent Unique Scan ID: '8c16b03b-8a0d-4dc6-ae52-936e6386497f'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 15:43:52.0 | 1053310862 |


|         2 | 15:43:52.9 | 1099380007 |


|         3 | 15:43:53.9 | 1131152533 |


...

|        28 | 15:44:18.9 | 1083187138 |


|        29 | 15:44:19.9 | 1054504915 |


|        30 | 15:44:20.9 | 1102493192 |


+-----------+------------+------------+
generator count ['8c16b03b'] (scan num: 5)





('8c16b03b-8a0d-4dc6-ae52-936e6386497f',)